In [ ]:
from google.colab import drive
drive.mount('/content/drive');

Mounted at /content/drive


In [ ]:
ROOT_PATH = "/content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto"

# Model

In [ ]:
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import layers
from keras.layers.core import Dense, Dropout

In [ ]:
SEQUENCE_LENGTH = 29
NUM_FEATURES = 1024

In [ ]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, output_dim, **kwargs):
        super().__init__(**kwargs)

        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim
        )

        self.sequence_length = sequence_length
        self.output_dim = output_dim

    def call(self, inputs):
        # The inputs are of shape: `(batch_size, frames, num_features)`
        length = tf.shape(inputs)[1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_positions = self.position_embeddings(positions)
        return inputs + embedded_positions

    def compute_mask(self, inputs, mask=None):
        mask = tf.reduce_any(tf.cast(inputs, "bool"), axis=-1)
        return mask

In [ ]:
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)

        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads

        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embed_dim,
            dropout=0.3
        )

        self.dense_proj = keras.Sequential([
            Dense(dense_dim, activation=tf.nn.gelu),
            Dense(embed_dim)
        ])

        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()

    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]

        attention_output = self.attention(inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)

In [ ]:
def build_transformer_model(
    dense_dim,
    num_heads,
    classes,
    dropout=0
):
    pos_emb = PositionalEmbedding(
        sequence_length=SEQUENCE_LENGTH,
        output_dim=NUM_FEATURES,
    )

    transf_enc = TransformerEncoder(
        embed_dim=NUM_FEATURES,
        dense_dim=dense_dim,
        num_heads=num_heads
    )

    head = Dense(classes, activation='softmax')

    inputs = keras.Input(shape=(None, None))
    embedded = pos_emb(inputs)

    x = transf_enc(embedded)
    x = layers.GlobalMaxPooling1D()(x)
    x = Dropout(dropout)(x)

    outputs = head(x)

    model = keras.Model(inputs, outputs)
    return model

In [ ]:
def train_model(
    model,
    train_data,
    train_labels,
    checkpoint_path,
    batch_size=32
):
    checkpoint = keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        save_weights_only=True,
        save_best_only=True,
        verbose=1
    )

    history = model.fit(
        train_data,
        train_labels,
        validation_split=VALIDATION_SPLIT, 
        epochs=EPOCHS,
        batch_size=batch_size, 
        callbacks=[checkpoint], 
        shuffle=True
    )

    return history

In [ ]:
def predict_labels(model, data):
    pred = model(data)
    return tf.math.argmax(pred, axis=1).numpy()

In [ ]:
# https://stackoverflow.com/questions/31324218/scikit-learn-how-to-obtain-true-positive-true-negative-false-positive-and-fal
def perf_measure(y_actual, y_hat):
    TP = 0
    FP = 0
    TN = 0
    FN = 0

    for i in range(len(y_hat)): 
        if y_actual[i] == y_hat[i] == 1:
           TP += 1
        if y_hat[i] == 1 and y_actual[i] != y_hat[i]:
           FP += 1
        if y_actual[i] == y_hat[i] == 0:
           TN += 1
        if y_hat[i] == 0 and y_actual[i] != y_hat[i]:
           FN += 1

    return(TP, FP, TN, FN)

# Data loading

In [ ]:
import numpy as np


SPLITTED_FEATURES_PATH = f"{ROOT_PATH}/features-and-labels"

with open(f"{SPLITTED_FEATURES_PATH}/train_labels.npy", 'rb') as file:
    raw_train_labels = np.squeeze(np.load(file))

with open(f"{SPLITTED_FEATURES_PATH}/test_labels.npy", 'rb') as file:
    raw_test_labels = np.squeeze(np.load(file))

with open(f"{SPLITTED_FEATURES_PATH}/train_features.npy", 'rb') as file:
    raw_train_features = np.squeeze(np.load(file))

with open(f"{SPLITTED_FEATURES_PATH}/test_features.npy", 'rb') as file:
    raw_test_features = np.squeeze(np.load(file))

---

In [ ]:
num_users, num_train_videos, num_frames, dims = raw_train_features.shape
_, num_test_videos, _, _ = raw_test_features.shape

In [ ]:
train_features = raw_train_features.reshape(num_users * num_train_videos, num_frames, dims)
train_features.shape

(960, 29, 1024)

In [ ]:
test_features = raw_test_features.reshape(num_users * num_test_videos, num_frames, dims)
test_features.shape

(320, 29, 1024)

In [ ]:
train_labels = raw_train_labels.reshape(num_users * num_train_videos, 1)
train_labels.shape

(960, 1)

In [ ]:
test_labels = raw_test_labels.reshape(num_users * num_test_videos, 1)
test_labels.shape

(320, 1)

In [ ]:
raw_train_features.shape
# users, videos, frames, features

(32, 30, 29, 1024)

# Training

In [ ]:
checkpoint_path = f"{ROOT_PATH}/models/transformer/checkpoints/base-model-weights.ckpt"

In [ ]:
dense_dim = 4096
num_heads = 1
classes = 2
dropout = 0.4

model = build_transformer_model(
    dense_dim, 
    num_heads, 
    classes
)

In [ ]:
ones = np.sum(train_labels)
zeros = len(train_labels) - ones

ones_to_zeros_ratio = ones / zeros

print(f"Ones to zeros ratio: {ones_to_zeros_ratio:.2f}")
print(f"Zeros to ones ratio: {1/ones_to_zeros_ratio:.2f}")
print()
print(f"Percentage of zeros: {zeros / (ones + zeros):.2f}")
print(f"Percentage of ones: {ones / (ones + zeros):.2f}")

Ones to zeros ratio: 1.31
Zeros to ones ratio: 0.76

Percentage of zeros: 0.43
Percentage of ones: 0.57


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
loss = "sparse_categorical_crossentropy",
metrics = ["accuracy"]

loss_weights = [ones_to_zeros_ratio, 1]
# loss_weights = [1, 1]

model.compile(
    optimizer=optimizer,
    loss=loss,
    metrics=metrics, 
    loss_weights=loss_weights
)

In [ ]:
EPOCHS = 15
VALIDATION_SPLIT = 0.1
BATCH_SIZE = 32

train_model(
    model,
    train_features,
    train_labels,
    checkpoint_path, 
    batch_size=BATCH_SIZE
)

Epoch 1/15
27/27 [==============================] - ETA: 0s - loss: 1.0041 - accuracy: 0.5370
Epoch 00001: val_loss improved from inf to 0.95193, saving model to /content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto/models/transformer/checkpoints/base-model-weights.ckpt
27/27 [==============================] - 4s 105ms/step - loss: 1.0041 - accuracy: 0.5370 - val_loss: 0.9519 - val_accuracy: 0.5104
Epoch 2/15
27/27 [==============================] - ETA: 0s - loss: 0.9169 - accuracy: 0.5544
Epoch 00002: val_loss did not improve from 0.95193
27/27 [==============================] - 2s 59ms/step - loss: 0.9169 - accuracy: 0.5544 - val_loss: 0.9592 - val_accuracy: 0.5833
Epoch 3/15
27/27 [==============================] - ETA: 0s - loss: 0.8752 - accuracy: 0.5995
Epoch 00003: val_loss improved from 0.95193 to 0.95027, saving model to /content/drive/Shareddrives/Proyecto RecSys 2021-2/Proyecto/models/transformer/checkpoints/base-model-weights.ckpt
27/27 [=============================

In [ ]:
preds = model(train_features)
preds

<tf.Tensor: shape=(960, 2), dtype=float32, numpy=
array([[0.45656237, 0.54343766],
       [0.3335815 , 0.6664185 ],
       [0.45925018, 0.54074985],
       ...,
       [0.40705234, 0.5929476 ],
       [0.40726608, 0.59273386],
       [0.57980096, 0.420199  ]], dtype=float32)>

In [ ]:
argmax = tf.math.argmax(preds, axis=1)
argmax

<tf.Tensor: shape=(960,), dtype=int64, numpy=
array([1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
     

In [ ]:
argmax_is_zero = argmax == 0
argmax_is_one = argmax == 1

In [ ]:
zero_prob = tf.reduce_mean(tf.boolean_mask(preds[:, 0], argmax_is_zero)).numpy()
one_prob = tf.reduce_mean(tf.boolean_mask(preds[:, 1], argmax_is_one)).numpy()

print(f"Mean zero prob when zero: {zero_prob:.2f}")
print(f"Mean one prob when one:   {one_prob:.2f}")

Mean zero prob when zero: 0.55
Mean one prob when one:   0.67


In [ ]:
print(f"Zeros: {sum(argmax_is_zero.numpy())}")
print(f"Ones:  {sum(argmax_is_one.numpy())}")

Zeros: 139
Ones:  821


---

# Evaluation

In [ ]:
model.load_weights(checkpoint_path);
loss, accuracy = model.evaluate(test_features, test_labels)

print(f"Test accuracy: {accuracy:.2f}")

10/10 [==============================] - 0s 28ms/step - loss: 0.9064 - accuracy: 0.5781
Test accuracy: 0.58


In [ ]:
num_test_samples, _, _ = test_features.shape

test_preds = model(test_features)
test_argmax = tf.math.argmax(test_preds, axis=1)

tf.reduce_sum(test_argmax).numpy() / num_test_samples

0.759375

In [ ]:
labels = tf.squeeze(test_labels).numpy()
preds = predict_labels(model, test_features)

In [ ]:
ones_in_preds = sum(preds)
zeros_in_preds = len(preds) - ones_in_preds

ones_in_labels = sum(labels)
zeros_in_labels = len(labels) - ones_in_labels

In [ ]:
print("Zeros in pred:", zeros_in_preds)
print("Zeros in labels:", zeros_in_labels)
print()
print("Ones in pred:", ones_in_preds)
print("Ones in labels:", ones_in_labels)

Zeros in pred: 77
Zeros in labels: 108

Ones in pred: 243
Ones in labels: 212


In [ ]:
tp, fp, tn, fn = perf_measure(labels, preds)

print("True positives:", tp)
print("False positives:", fp)
print()
print("True negatives:", tn)
print("False negatives:", fn)

True positives: 160
False positives: 83

True negatives: 25
False negatives: 52
